In [1]:
import asdf
import os
import numpy as np

from dataclasses import dataclass
from pathlib import Path

@dataclass
class Ellipse:
    semi_major: float
    semi_minor: float

ellipse_uri = "asdf://example.com/example-project/schemas/ellipse-1.0.0"

ellipse_schema_content = f"""
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01
id: {ellipse_uri}

type: object
properties:
  semi_major:
    type: number
  semi_minor:
    type: number
required: [semi_major, semi_minor]
...
"""

asdf.get_config().add_resource_mapping({ellipse_uri: ellipse_schema_content})

shapes_manifest_uri = "asdf://example.com/example-project/manifests/shapes-1.0.0"
shapes_extension_uri = "asdf://example.com/example-project/extensions/shapes-1.0.0"
ellipse_tag = "asdf://example.com/example-project/tags/ellipse-1.0.0"

class EllipseConverter(asdf.extension.Converter):
    tags = [ellipse_tag]
    types = [Ellipse]

    def to_yaml_tree(self, obj, tag, ctx):
        return {
            "semi_major": obj.semi_major,
            "semi_minor": obj.semi_minor,
        }

    def from_yaml_tree(self, node, tag, ctx):
        return Ellipse(semi_major=node["semi_major"], semi_minor=node["semi_minor"])

@dataclass
class RotatedEllipse(Ellipse):
    position_angle: float

rotated_ellipse_uri = "asdf://example.com/example-project/schemas/rotated_ellipse-1.0.0"

rotated_ellipse_schema_content = f"""
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01
id: {rotated_ellipse_uri}

allOf:
  - $ref: {ellipse_uri}
  - properties:
      position_angle:
        type: number
    required: [position_angle]
...
"""

asdf.get_config().add_resource_mapping(
    {rotated_ellipse_uri: rotated_ellipse_schema_content}
)

rotated_ellipse_tag = "asdf://example.com/example-project/tags/rotated_ellipse-1.0.0"

class RotatedEllipseConverter(asdf.extension.Converter):
    tags = [ellipse_tag, rotated_ellipse_tag]
    types = [Ellipse, RotatedEllipse]

    def select_tag(self, obj, tag, ctx):
        if isinstance(obj, RotatedEllipse):
            return rotated_ellipse_tag
        elif isinstance(obj, Ellipse):
            return ellipse_tag
        else:
            raise ValueError(f"Unknown object {type(obj)}")

    def to_yaml_tree(self, obj, tag, ctx):
        tree = {
            "semi_major": obj.semi_major,
            "semi_minor": obj.semi_minor,
        }

        if tag == rotated_ellipse_tag:
            tree["position_angle"] = obj.position_angle

        return tree

    def from_yaml_tree(self, node, tag, ctx):
        if tag == ellipse_tag:
            return Ellipse(**node)
        elif tag == rotated_ellipse_tag:
            return RotatedEllipse(**node)
        else:
            raise ValueError(f"Unknown tag {tag}")

# Exercise 1

In [2]:
@dataclass
class Rectangle:
    base: float
    height: float

In [3]:
# Create
rectangle_uri = "asdf://example.com/example-project/schemas/rectangle-1.0.0"

rectangle_schema_content = f"""
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01
id: {rectangle_uri}

type: object
properties:
  base:
    type: number
  height:
    type: number
required: [base, height]
...
"""

# Add
asdf.get_config().add_resource_mapping({rectangle_uri: rectangle_schema_content})

# Check
schema = asdf.schema.load_schema(rectangle_uri)
asdf.schema.check_schema(schema)
test_rectangle_object = {"base": 1.0, "height": 2.0}
asdf.schema.validate(test_rectangle_object, schema=schema)

# Exercise 2

In [4]:
rectangle_tag = "asdf://example.com/example-project/tags/rectangle-1.0.0"

shapes_manifest_content = f"""
%YAML 1.1
---
id: {shapes_manifest_uri}
extension_uri: {shapes_extension_uri}

title: Example Shape extension 1.0.0
description: Tags for example shape objects.

tags:
  - tag_uri: {ellipse_tag}
    schema_uri: {ellipse_uri}

  - tag_uri: {rectangle_tag}
    schema_uri: {rectangle_uri}
...
"""

asdf.get_config().add_resource_mapping({shapes_manifest_uri: shapes_manifest_content})

# check
schema = asdf.schema.load_schema(shapes_manifest_uri)
asdf.schema.check_schema(schema)
asdf.schema.validate(shapes_manifest_content, schema=schema)

# Exercise 3

In [5]:
class RectangleConverter(asdf.extension.Converter):
    tags = [rectangle_tag]
    types = [Rectangle]

    def to_yaml_tree(self, obj, tag, ctx):
        return {
            "base": obj.base,
            "height": obj.height,
        }

    def from_yaml_tree(self, node, tag, ctx):
        return Rectangle(base=node["base"], height=node["height"])

# Exercise 4

In [6]:
shapes_extension = asdf.extension.ManifestExtension.from_uri(
    shapes_manifest_uri, converters=[EllipseConverter(), RectangleConverter()]
)
asdf.get_config().add_extension(shapes_extension)

In [7]:
# Check

ellipse = Ellipse(1.0, 2.0)
rectangle = Rectangle(3.0, 4.0)

with asdf.AsdfFile() as af:
    af["ellipse"] = ellipse
    af["rectangle"] = rectangle
    af.write_to("exercise_4.asdf")

with open("exercise_4.asdf") as f:
    print(f.read())

with asdf.open("exercise_4.asdf") as af:
    print(af["ellipse"])
    assert af["ellipse"] == ellipse
    print(af["rectangle"])
    assert af["rectangle"] == rectangle

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
  - !core/extension_metadata-1.0.0 {extension_class: asdf.extension._manifest.ManifestExtension,
    extension_uri: 'asdf://example.com/example-project/extensions/shapes-1.0.0'}
ellipse: !<asdf://example.com/example-project/tags/ellipse-1.0.0> {semi_major: 1.0,
  semi_minor: 2.0}
rectangle: !<asdf://example.com/example-project/tags/rectangle-1.0.0> {base: 3.0,
  height: 4.0}
...

Ellipse(semi_major=1.0, semi_minor=2.0)
Rectangle(base=3.0, height=4.0)


# Exercise 5

In [8]:
@dataclass
class RectangularPrism(Rectangle):
    depth: float

In [9]:
rectangular_prism_uri = "asdf://example.com/example-project/schemas/rectangular_prism-1.0.0"

rectangular_prism_schema_content = f"""
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01
id: {rectangular_prism_uri}

allOf:
  - $ref: {rectangle_uri}
  - properties:
      depth:
        type: number
    required: [depth]
...
"""

asdf.get_config().add_resource_mapping(
    {rectangular_prism_uri: rectangular_prism_schema_content}
)

# check
schema = asdf.schema.load_schema(rectangular_prism_uri)
asdf.schema.check_schema(schema)

test_rectangular_prism_object = {
    "base": 1.0,
    "height": 2.0,
    "depth": 3.0,
}
asdf.schema.validate(test_rectangular_prism_object, schema=schema)

# Exercise 6

In [10]:
rectangular_prism_tag = "asdf://example.com/example-project/tags/rectangular_prism-1.0.0"

shapes_manifest_content = f"""
%YAML 1.1
---
id: {shapes_manifest_uri}
extension_uri: {shapes_extension_uri}

title: Example Shape extension 1.0.0
description: Tags for example shape objects.

tags:
  - tag_uri: {ellipse_tag}
    schema_uri: {ellipse_uri}

  - tag_uri: {rotated_ellipse_tag}
    schema_uri: {rotated_ellipse_uri}

  - tag_uri: {rectangle_tag}
    schema_uri: {rectangle_uri}

  - tag_uri: {rectangular_prism_tag}
    schema_uri: {rectangular_prism_uri}
...
"""

asdf.get_config().add_resource_mapping(
    {shapes_manifest_uri: shapes_manifest_content}
)


# check
schema = asdf.schema.load_schema(shapes_manifest_uri)
asdf.schema.check_schema(schema)
asdf.schema.validate(shapes_manifest_content, schema=schema)

# Exercise 7

In [11]:
class RectangularPrismConverter(asdf.extension.Converter):
    tags = [rectangle_tag, rectangular_prism_tag]
    types = [Rectangle, RectangularPrism]

    def select_tag(self, obj, tag, ctx):
        if isinstance(obj, RectangularPrism):
            return rectangular_prism_tag
        elif isinstance(obj, Rectangle):
            return rectangle_tag
        else:
            raise ValueError(f"Unknown object {type(obj)}")

    def to_yaml_tree(self, obj, tag, ctx):
        tree = {
            "base": obj.base,
            "height": obj.height,
        }

        if tag == rectangular_prism_tag:
            tree["depth"] = obj.depth

        return tree

    def from_yaml_tree(self, node, tag, ctx):
        if tag == rectangle_tag:
            return Rectangle(**node)
        elif tag == rectangular_prism_tag:
            return RectangularPrism(**node)
        else:
            raise ValueError(f"Unknown tag {tag}")

In [12]:
shapes_extension = asdf.extension.ManifestExtension.from_uri(
    shapes_manifest_uri, converters=[RotatedEllipseConverter(), RectangularPrismConverter()]
)
asdf.get_config().add_extension(shapes_extension)

In [13]:
# Check

ellipse = Ellipse(1.0, 2.0)
rotated_ellipse = RotatedEllipse(1.0, 2.0, 3.0)

rectangle = Rectangle(3.0, 4.0)
rectangular_prism = RectangularPrism(3.0, 4.0, 5.0)

with asdf.AsdfFile() as af:
    af["ellipse"] = ellipse
    af["rotated_ellipse"] = rotated_ellipse
    af["rectangle"] = rectangle
    af["rectangular_prism"] = rectangular_prism
    af.write_to("exercise_8.asdf")

# Check
with open("exercise_8.asdf") as f:
    print(f.read())

with asdf.open("exercise_8.asdf") as af:
    print(af["ellipse"])
    assert af["ellipse"] == ellipse

    print(af["rotated_ellipse"])
    assert af["rotated_ellipse"] == rotated_ellipse

    print(af["rectangle"])
    assert af["rectangle"] == rectangle

    print(af["rectangular_prism"])
    assert af["rectangular_prism"] == rectangular_prism

#ASDF 1.0.0
#ASDF_STANDARD 1.5.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.1.0
asdf_library: !core/software-1.0.0 {author: The ASDF Developers, homepage: 'http://github.com/asdf-format/asdf',
  name: asdf, version: 2.12.0}
history:
  extensions:
  - !core/extension_metadata-1.0.0
    extension_class: asdf.extension.BuiltinExtension
    software: !core/software-1.0.0 {name: asdf, version: 2.12.0}
  - !core/extension_metadata-1.0.0 {extension_class: asdf.extension._manifest.ManifestExtension,
    extension_uri: 'asdf://example.com/example-project/extensions/shapes-1.0.0'}
ellipse: !<asdf://example.com/example-project/tags/ellipse-1.0.0> {semi_major: 1.0,
  semi_minor: 2.0}
rectangle: !<asdf://example.com/example-project/tags/rectangle-1.0.0> {base: 3.0,
  height: 4.0}
rectangular_prism: !<asdf://example.com/example-project/tags/rectangular_prism-1.0.0> {
  base: 3.0, depth: 5.0, height: 4.0}
rotated_ellipse: !<asdf://example.com/example-project/tags/rotated_ellipse-1.0.0> {
 